In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.workflow import FunctionAgent, AgentStream
from llama_index.core.workflow import (
    Context, InputRequiredEvent, HumanResponseEvent, JsonSerializer
)

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
async def dangerous_task(ctx: Context) -> str:
    """A dangerous task that requires human confirmation."""
    
    question = "Are you sure you want to proceed?"
    response = await ctx.wait_for_event(
        HumanResponseEvent,
        waiter_id=question,
        waiter_event=InputRequiredEvent(
            prefix=question,
            user_name="Titus"
        ),
        requirements={"user_name": "Titus"}
    )
    
    if response.response.strip().lower() == "yes":
        return "Dangerous task completed successfully."
    else:
        return "Dangerous task aborted"

In [6]:
llm = Ollama(model="qwen2.5", temperature=0)
    
agent = FunctionAgent(
    llm=llm,
    tools=[dangerous_task],
    system_prompt="You are a helpful assistant that can perform dangerous tasks with human approval."
)

In [7]:
ctx = Context(agent)
query = "I want to proceed with the dangerous task."
handler = agent.run(query, ctx=ctx)
input_ev = None

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
    if isinstance(event, InputRequiredEvent):
        print(event.prefix)
        input_ev = event
        break

ctx_dict = handler.ctx.to_dict(serializer=JsonSerializer())

restored_ctx = Context.from_dict(agent, ctx_dict, serializer=JsonSerializer())
handler = agent.run(ctx=restored_ctx)
handler.ctx.send_event(
    HumanResponseEvent(
        response="yes",
        user_name=input_ev.user_name,
    )
)
async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
    if isinstance(event, InputRequiredEvent):
        print(event.prefix)
        input_ev = event
        break

response = await handler
print(str(response))

Are you sure you want to proceed?
The dangerous task has been completed successfully. Please note that such tasks should always be performed with caution and under proper supervision.The dangerous task has been completed successfully. Please note that such tasks should always be performed with caution and under proper supervision.
